In [26]:
import pandas as pd
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from matplotlib.lines import Line2D
from collections import Counter
from nameparser import HumanName
import gender_guesser.detector as gender
from scipy.stats import chi2_contingency
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np


In [27]:
# File Paths
absolute_path = "C:\\Users\\tykun\\\OneDrive\\Documents\\SchoolDocs\VSCodeProjects\\connectedData\\board_analysis\\"
altered_dataframes = "altered_dataframes\\"
gpt_dataframes = "gpt_dataframes\\"
graphs = "graphs\\"
scripts =  "scripts\\"
board_dataframes = "board_dataframes\\"
college_matching = "college_matching\\"

altered_dataframe_path = f"{absolute_path}{altered_dataframes}"
gpt_dataframe_path = f"{absolute_path}{gpt_dataframes}" 
graph_path = f"{absolute_path}{graphs}"
script_path = f"{absolute_path}{scripts}"
boards_path = f"{absolute_path}{board_dataframes}"
matching_path = f"{absolute_path}{college_matching}"


# Valid Years
years = ["1999", "2000", "2005", "2008", "2009", "2013"]

In [28]:
academic_programs_df = pd.read_csv(f"{matching_path}university_academics_program_percent.csv")
university_board_statistics_df = pd.read_csv(f"{altered_dataframe_path}sample_board_statistics.csv")

In [29]:
print("univesity_board_statistics_df: " , university_board_statistics_df.columns)
print("academic_programs_df: ", academic_programs_df.columns)

univesity_board_statistics_df:  Index(['Year', 'Institution', 'carnegie_id', 'AffiliationId',
       'female_president', 'poc_president', 'total_members', 'poc', 'white',
       'female', 'male', 'unknown', 'poc_change', 'white_change',
       'unknown_change', 'male_change', 'female_change', 'StateSystem'],
      dtype='object')
academic_programs_df:  Index(['carnegie_id', 'AffiliationId', 'year',
       'academics.program_percentage.agriculture',
       'academics.program_percentage.resources',
       'academics.program_percentage.architecture',
       'academics.program_percentage.ethnic_cultural_gender',
       'academics.program_percentage.communication',
       'academics.program_percentage.communications_technology',
       'academics.program_percentage.computer',
       'academics.program_percentage.personal_culinary',
       'academics.program_percentage.education',
       'academics.program_percentage.engineering',
       'academics.program_percentage.engineering_technology',

In [33]:
# Calculate the total change in female representation and the total members per institution
institution_year_counts = university_board_statistics_df.groupby('Institution').size().reset_index(name='year_count')
female_change_sum = university_board_statistics_df.groupby(['Institution', 'AffiliationId'])['female_change'].sum().reset_index()
total_members_change_sum = university_board_statistics_df.groupby('Institution')['total_members'].sum().reset_index(name='total_members_sum')

# Filter to include only institutions that appear in at least 5 years of data
institution_year_counts = institution_year_counts[institution_year_counts['year_count'] >= 5]

# Merge data to calculate the normalized change per year, only including valid institutions
avg_female_change = pd.merge(female_change_sum, institution_year_counts, on='Institution')
avg_female_change = pd.merge(avg_female_change, total_members_change_sum, on='Institution')

# Calculate the average female change normalized by total members change
avg_female_change['avg_female_change_per_member'] = avg_female_change['female_change'] / avg_female_change['total_members_sum']

# Sort by the highest average change per member
avg_female_change = avg_female_change.sort_values(by='avg_female_change_per_member', ascending=False)

# Keep the top 10 institutions with the highest average change normalized by total members
top_institutions_avg_female_trend = avg_female_change.head(10)

# Print the results
print("Top Institutions with the Highest Normalized Average Yearly Change in Female Board Members:")
print(top_institutions_avg_female_trend)

Top Institutions with the Highest Normalized Average Yearly Change in Female Board Members:
                                           Institution  AffiliationId  \
90                             Texas Womans University    150913223.0   
126  Virginia Polytechnic Institute And State Unive...    859038795.0   
130                             Wayne State University    185443292.0   
60                   Michigan Technological University     11957088.0   
102                             University Of Michigan     27837315.0   
87                                 Syracuse University     70983195.0   
61                        Mississippi State University     99041443.0   
83                       South Dakota State University    177156846.0   
104                    University Of Northern Colorado     71966907.0   
21                                 Clarkson University     16944753.0   

     female_change  year_count  total_members_sum  \
90             3.0           6               35.0  

In [36]:
# Extract the top institutions' AffiliationId
top_affiliation_ids = top_institutions_avg_female_trend['AffiliationId'].tolist()

# Filter academic_programs_df to only include the top institutions by AffiliationId
filtered_academic_programs_df = academic_programs_df[academic_programs_df['AffiliationId'].isin(top_affiliation_ids)]

# List of columns representing program percentages
program_columns = [col for col in academic_programs_df.columns if 'program_percentage' in col]

# Calculate the average yearly change in program percentages for each AffiliationId over the years
program_trends = filtered_academic_programs_df.groupby('AffiliationId')[program_columns].apply(
    lambda x: x.diff().mean()
).reset_index()

# Filter the program columns where there is a positive average change to identify increasing programs
increasing_programs = program_trends.loc[:, (program_trends > 0).any()]

# Display the AffiliationId and the columns with positive average changes
increasing_programs_with_positive_trend = increasing_programs[
    ['AffiliationId'] + [col for col in increasing_programs.columns if col.startswith('academics.program_percentage')]
]


# Print the filtered DataFrame for verification
print("Academic programs with increasing funding for top institutions:")
print(increasing_programs_with_positive_trend)

Academic programs with increasing funding for top institutions:
   AffiliationId  academics.program_percentage.agriculture  \
0     11957088.0                                  0.000000   
1     16944753.0                                  0.000000   
2     27837315.0                                  0.000000   
3     70983195.0                                  0.000000   
4     71966907.0                                  0.000000   
5     99041443.0                                  0.000560   
6    150913223.0                                  0.000000   
7    177156846.0                                 -0.000515   
8    185443292.0                                  0.000000   
9    859038795.0                                 -0.000650   

   academics.program_percentage.resources  \
0                                0.000495   
1                               -0.000300   
2                               -0.000105   
3                                0.000000   
4                           